In [1]:
# Hidden cell for nbsphinx (check cell metadata)
import warnings
warnings.simplefilter('ignore')

# Create TARDIS atomic files

This notebook provides a quick introduction in how to use Carsus with the SQL interface.

## Creating a database

To start working with a database you need to initialize it. 

This requires an `url`, for example `sqlite:///path/to/example.db`. In this quickstart, we'll use a in-memory `sqlite` database, so we leave the `url` empty:

In [2]:
from carsus import init_db
session = init_db()

Initializing the database at sqlite://
Ingesting basic atomic data


Now, we have a SQLAlchemy `Session` object named `session`. We can use `session` to make simple queries. As you can see from the output, some "basic atomic data" has been ingested into our database. Let's examine it:

In [3]:
from carsus.model import Atom

print('Number of Atoms in the database: {}'.format(session.query(Atom).count()))
si = session.query(Atom).get(14) 
print("Symbol: {}, atomic_number: {}, name: {}".format(si.symbol, si.atomic_number, si.name))

Number of Atoms in the database: 118
Symbol: Si, atomic_number: 14, name: Silicon


So, our database already contains basic information about atoms. But this informaition has not been commited to the database yet. It is always **your** responsibility to commit data!

In [4]:
session.commit()

To store more interesting data we are going to use ingesters. For each data source supported by Carsus there is a corresponding ingester class. For example, let's ingest atomic weights from the NIST Atomic Weights and Isotopic Compositions database:

In [5]:
from carsus.io.nist import NISTWeightsCompIngester

weightscomp_ingester = NISTWeightsCompIngester(session)
weightscomp_ingester.ingest()
session.commit()

Ingesting atomic weights from nist


And now atoms have atomic weights:

In [6]:
print(si.weights)

[<Quantity: 28.085 u>]


Other ingesters are just the same, but they often take specific arguments:

In [7]:
from carsus.io.nist import NISTIonizationEnergiesIngester

# Ingest ionization energies and ground levels from the NIST Atomic Spectra Database
ioniz_energies_ingester = NISTIonizationEnergiesIngester(session, spectra="H-Si")
ioniz_energies_ingester.ingest(ionization_energies=True, ground_levels=True)
session.commit()

Ingesting ionization energies from nist-asd
Ingesting ground levels from nist-asd


**IMPORTANT:** it's necessary ingest NIST ionization energies before using `GFALLIngester`

In [8]:
!wget -q http://kurucz.harvard.edu/linelists/gfall/gfall.dat

In [9]:
from carsus.io.kurucz import GFALLIngester

# Ingest levels and lines from the Kurucz line list
gfall_ingester = GFALLIngester(session, fname="./gfall.dat", ions="H-Si")
gfall_ingester.ingest(levels=True, lines=False)
session.commit()

[carsus.io.kurucz.gfall][WARNING]  A specific combination to identify unique levels from the gfall data has not been given. Defaulting to ["energy", "j"]. (gfall.py:68)
[carsus.io.kurucz.gfall][INFO   ]  Parsing GFALL ./gfall.dat (gfall.py:116)
Ingesting levels from ku_latest
Ingesting levels for He 0
Ingesting levels for He 1
Ingesting levels for Li 0
Ingesting levels for Li 1
Ingesting levels for Be 0
Ingesting levels for Be 1
Ingesting levels for Be 2
Ingesting levels for B 0
Ingesting levels for B 1
Ingesting levels for B 2
Ingesting levels for B 3
Ingesting levels for C 0
Ingesting levels for C 1
Ingesting levels for C 2
Ingesting levels for C 3
Ingesting levels for N 0
Ingesting levels for N 1
Ingesting levels for N 2
Ingesting levels for N 3
Ingesting levels for N 4
Ingesting levels for N 5
Ingesting levels for O 0
Ingesting levels for O 1
Ingesting levels for O 2
Ingesting levels for O 3
Ingesting levels for O 4
Ingesting levels for O 5
Ingesting levels for F 0
Ingesting levels

In [10]:
from carsus.io.chianti_ import ChiantiIngester

# Ingest levels and lines from Chianti database
chianti_ingester = ChiantiIngester(session, ions="Si 1-2")
chianti_ingester.ingest(levels=True, lines=True, collisions=True)
session.commit()

 ChiantiPy version 0.8.4 
 found PyQt5 widgets
 using PyQt5 widgets
Ingesting levels from chianti_v9.0.1
Ingesting levels for Si 1
Ingesting levels for Si 2
Ingesting lines from chianti_v9.0.1
Ingesting lines for Si 1
Ingesting lines for Si 2
Ingesting collisions from chianti_v9.0.1
Ingesting collisions for Si 1
Ingesting collisions for Si 2


In [11]:
import os
import carsus
from carsus.io.zeta import KnoxLongZetaIngester

CARSUS_PATH = os.path.dirname(carsus.__file__)

# Ingest Knox Long's recombination coefficients
zeta_ingester = KnoxLongZetaIngester(session, os.path.join(CARSUS_PATH, 'data/knox_long_recombination_zeta.dat'))
zeta_ingester.ingest()
session.commit()

Now we have ionization energies, levels and lines for ions of Si (Note: Si 1 = Si II, Si 2 = Si III, etc.). You should **always** use arabic numerals when specifying ions:

In [12]:
from carsus.model import Ion

si_1 = session.query(Ion).get((14,1))
print(len(si_1.levels))

190


This is it! Now you can explore the your database or create output HDF5 files.

## Creating output HDF5 files

Once you have a database it is very easy to create atomic data files for TARDIS. 
To do this, you need to use a special class called `AtomData`. The class takes `session` as its first argument; other important parameters are `selected_atoms` and `chianti_ions`. Only data for `selected_atoms` will be stored in the output DataFrames and levels and lines for `chianti_ions` will be taken from the data source with the same name. Let's create an `AtomData` instance: 

In [13]:
from carsus.io.output import AtomData
atom_data = AtomData(session, 
                     selected_atoms="Si", 
                     chianti_short_name='chianti_v9.0.1', 
                     chianti_ions="Si 1-2")

**IMPORTANT:** if `chianti_short_name` doesn't match with your current version of Chianti DB the above command will fail.  

The output DataFrames can be easily accessed as "prepared" attributes of `atom_data`:

In [14]:
print(atom_data.atom_masses_prepared)

              symbol        name    mass
atomic_number                           
14             b'Si'  b'Silicon'  28.085


In [15]:
print(atom_data.levels_prepared.head(20))

                                         energy  g  metastable
atomic_number ion_number level_number                         
14            0          0             0.000000  1        True
                         1             0.009561  3        True
                         2             0.027668  5        True
                         3             0.780958  5        True
                         4             1.908659  1        True
                         5             4.131904  5        True
                         6             4.920085  1        True
                         7             4.929647  3        True
                         8             4.953795  5        True
                         9             5.082346  3        True
                         10            5.613532  3        True
                         11            5.615693  5        True
                         12            5.615694  5        True
                         13            5.615694  5     

In [16]:
print(atom_data.macro_atom_prepared.head(20))

    atomic_number  ion_number  source_level_number  destination_level_number  \
0              14           1                    0                         5   
1              14           1                    0                         7   
2              14           1                    0                         8   
3              14           1                    0                         9   
4              14           1                    0                        13   
5              14           1                    0                        14   
6              14           1                    0                        15   
7              14           1                    0                        16   
8              14           1                    0                        24   
9              14           1                    0                        25   
10             14           1                    1                         7   
11             14           1           

To create an HDF5 file you need to use the `to_hdf()` method. You should specify every DataFrame that you want to store:

In [17]:
atom_data.to_hdf("example_store.h5", 
                 store_atom_masses=True, 
                 store_ionization_energies=True, 
                 store_levels=True, 
                 store_lines=True,
                 store_collisions=True,
                 store_macro_atom=True,
                 store_zeta_data=True)

Signing AtomData: 
MD5: 0bec3e488809cf5125ece35a49bdb739
UUID1: f0101a2a823811ea9991c83dd46c2c53


You are done! Now you can use the created HDF5 atomic file to run TARDIS simulations.